In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import csv
import os
warnings.filterwarnings('ignore')
from datetime import datetime

In [2]:
pd.options.display.float_format = '{:.2f}'.format

In [3]:
directory = r'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES OP\DEC'

# Trouver le fichier CSV le plus récent dans le répertoire
def get_latest_file(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    latest_file = max(files, key=os.path.getctime)
    return latest_file

# Charger le fichier CSV le plus récent
latest_file_path = get_latest_file(directory)
cinetpay = pd.read_csv(latest_file_path,sep=';')
cinetpay.sample()



directory = r'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES PMT\DEC'

# Trouver le fichier CSV le plus récent dans le répertoire
def get_latest_file(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    latest_file = max(files, key=os.path.getctime)
    return latest_file

# Charger le fichier CSV le plus récent
latest_file_path = get_latest_file(directory)
pmt = pd.read_csv(latest_file_path)
pmt.sample()

,created_at,payment_date,operator,merchant_name,transaction_id,merchant_transaction_id,id_operator,external_transaction_id,amount,fee_amount,merchant_amount,operation_origin,currency,statut,country,provider_id,provider_name,phone_number
14305,2024-12-16 13:03:01,NaN,AIRTEL_CD,1XBET CD,dbae4a96-1532-4c67-8936-7944f3574f04,3048452271,EA241216.130317.V675641,NaN,20000.00,1000.00,21000.00,transfer,CDF,SUCCESS,CD,EA241216.130317.V675641@@@@@dbae4a96-1532-4c67...,CinetPay_Payout_AIRTEL_CD,243978249345


In [4]:
#cinetpay = pd.read_csv(r'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES OP\JUILLET\20240715_cinetpay_transfer_20240708_20240714.csv', sep=';')

In [5]:
#pmt = pd.read_csv(r'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\DONNEES PMT\JUILLET\E20240714_PMT_Cinetpay_20240708_20240714.csv', on_bad_lines='skip')

In [6]:
#SAUVEGARDER UNE COPIE DE LA DATAFRAME
dfop1=cinetpay.copy()
dfpmt1=pmt.copy()

In [7]:
cinetpay1=cinetpay[['Date Création (GMT)',
            'Date Modification',
            'Date de traitement',
           'ID Marchand',
           'ID Operateur',
           'Téléphone',
            'Opérateur',
           'Montant Envoyé',
           'Statut',
           'Commentaire'
]]

In [8]:
dfpmt = pmt.rename(columns={
        'created_at': 'Created Date',
        'payment_date': 'Payment Date',
        'operator': 'Operator',
        'merchant_name': 'Merchant Name',
        'transaction_id': 'Transaction ID',
        'id_operator': 'ID Opérateur',
        'country':'Pays',
        'phone_number': 'Phone Number',
        'amount': 'Montant',
        'fee_amount': 'Fee amount',
        'merchant_amount': 'Merchant amount',
        'statut': 'Statut'
    })

In [9]:
dfpmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19180 entries, 0 to 19179
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Created Date             19180 non-null  object 
 1   Payment Date             0 non-null      float64
 2   Operator                 19180 non-null  object 
 3   Merchant Name            19180 non-null  object 
 4   Transaction ID           19180 non-null  object 
 5   merchant_transaction_id  19180 non-null  object 
 6   ID Opérateur             19117 non-null  object 
 7   external_transaction_id  0 non-null      float64
 8   Montant                  19180 non-null  float64
 9   Fee amount               19180 non-null  float64
 10  Merchant amount          19180 non-null  float64
 11  operation_origin         19180 non-null  object 
 12  currency                 19180 non-null  object 
 13  Statut                   19180 non-null  object 
 14  Pays                  

In [10]:
#EXTRAITRE LA DATE DANS UNE NOUVELLE COLLONNE
dfpmt['DateCourte'] = dfpmt['Created Date'].apply(lambda x: x.split(' ')[0])
cinetpay['DateCourte'] = cinetpay1['Date Création (GMT)'].apply(lambda x: x.split(' ')[0])

In [11]:
cinetpay.Statut.unique()

array(['VAL', 'REJ'], dtype=object)

In [12]:
new_rec= cinetpay.loc[(cinetpay['Statut'] == 'NEW') |(cinetpay['Statut'] == 'REC') ]

In [13]:
rej= cinetpay.loc[(cinetpay['Statut'] == 'REJ')]

In [14]:
#MISE EN PLACE DE RECHERCHE X POUR RECUPERATION CHEZ L'OP
        # Vérification des correspondances entre A1 et B1
correspondance_statut_op= cinetpay.set_index('ID Marchand')['Statut']
correspondance_date_op = cinetpay.set_index('ID Marchand')['DateCourte']
correspondance_op = cinetpay.set_index('ID Marchand')['Opérateur']
correspondance_idoperator = cinetpay.set_index('ID Marchand')['ID Operateur']


dfpmt['DATEOP'] = dfpmt['Transaction ID'].map(correspondance_date_op)
dfpmt['STATUTOP'] = dfpmt['Transaction ID'].map(correspondance_statut_op)
dfpmt['OPERATOROP'] = dfpmt['Transaction ID'].map(correspondance_op)
dfpmt['IDOPERATOR'] = dfpmt['Transaction ID'].map(correspondance_idoperator)

In [15]:
#CONVERTIR EN FLOAT
#dfpmt['Montant'] = dfpmt['Montant'].str.replace(',', '.').astype(float)
#dfpmt['Fee amount'] = dfpmt['Fee amount'].str.replace(',', '.').astype(float)
#dfpmt['Merchant amount'] = dfpmt['Merchant amount'].str.replace(',', '.').astype(float)

In [16]:
# Définir les taux de commission pour chaque opérateur
commission_rates = {
        'AIRTELCD' : 0.018,
        'MPESACD': 0.028,
        'MOOVBF': 0.01,
        'OMCD':	0.01,
        'OMML':	0.008,
        'MOOVML': 0.008,
        'OMBF':	0.01,
        'OM': 0.008,
        'FLOOZ': 0.007,
        'FLOOZTG': 0.0080,
        'TMONEYTG': 0.0080,
        'MOMO':	0.0050,
        'MLTEL': 0.0080
         

}

# Fonction pour calculer les frais d'opérateur
def calculate_frais_op(row):
    operator = row['OPERATOROP']
    montant = row['Montant']
    commission_rate = commission_rates.get(operator, 0)
    return montant * commission_rate

# Ajouter une colonne 'frais_op' avec les frais d'opérateur calculés
dfpmt['frais_op'] = dfpmt.apply(calculate_frais_op, axis=1)

In [17]:
dfpmt['Frais_pmt'] = dfpmt['Fee amount'] - dfpmt['frais_op']

In [18]:
dfop= cinetpay.loc[(cinetpay['Statut'] == 'VAL')]

In [19]:
#NBSI PMT &CINETPAY
dfpmt['CINETPAY'] = dfpmt['Transaction ID'].isin(dfop['ID Marchand']).astype(int)
dfop['PMT'] = dfop['ID Marchand'].isin(dfpmt['Transaction ID']).astype(int)
dfpmt['NEW_REC'] = dfpmt['Transaction ID'].isin(new_rec['ID Marchand']).astype(int)
dfpmt['REJ'] = dfpmt['Transaction ID'].isin(rej['ID Marchand']).astype(int)

In [20]:
dfpmt['Nombre']= dfpmt['Montant']
dfop['Nombre']=dfop['Montant Envoyé']

In [21]:
df_filteredpmt = dfpmt[dfpmt['CINETPAY'] == 1]

# Création du tableau croisé dynamique
tcdpmt = pd.pivot_table(
    df_filteredpmt,
    values=['Montant', 'Nombre','frais_op', 'Frais_pmt'],
    index=['DateCourte', 'OPERATOROP','Statut'],
    aggfunc={'Nombre': 'count','Montant': 'sum' ,'frais_op': 'sum', 'Frais_pmt': 'sum' },
    fill_value=0,
    margins=True,
    margins_name='Total')


df_filtered = dfop[(dfop['PMT'] == 1) | (dfop['PMT'] == 0)]

# Création du tableau croisé dynamique
tcdcinetpay = pd.pivot_table(
    df_filtered,
    values=['Montant Envoyé', 'Nombre'],
    index=['DateCourte', 'Opérateur','Statut'],
    aggfunc={'Nombre': 'count','Montant Envoyé': 'sum' },
    fill_value=0,
    margins=True,
    margins_name='Total'
)

In [22]:
tcdcinetpay

Montant Envoyé  Nombre
DateCourte Opérateur Statut                        
2024-12-16 AIRTELCD  VAL           18143840     573
           FLOOZ     VAL              26030       3
           FLOOZTG   VAL            1316845     207
           MOMO      VAL              67000       5
           MOOVBF    VAL           10025640     823
           MPESACD   VAL           25531400     315
           OM        VAL           23480470    1758
           OMBF      VAL           28543070    2482
           OMCD      VAL            3608300      43
           OMSN      VAL             149000       2
           TMONEYTG  VAL            5002970     551
Total                             115894565    6762

In [23]:
tcdpmt

Frais_pmt      Montant  Nombre   frais_op
DateCourte OPERATOROP Statut                                            
2024-12-16 AIRTELCD   SUCCESS  540910.70  18143845.00     573  326589.21
           FLOOZ      SUCCESS    1010.49     26030.00       3     182.21
           FLOOZTG    SUCCESS   41133.19   1316845.00     207   10534.76
           MOMO       SUCCESS    5695.00     67000.00       5     335.00
           MOOVBF     PENDING     900.00     20000.00       1     200.00
                      SUCCESS  433140.04  10005651.00     822  100056.51
           MPESACD    SUCCESS  517311.14  25531413.00     315  714879.56
           OM         PENDING      90.00      7500.00       1      60.00
                      SUCCESS  761312.90  23472989.00    1757  187783.91
           OMBF       PENDING    1330.00     27000.00       4     270.00
                      SUCCESS 1173698.92  28516130.00    2478  285161.30
           OMCD       SUCCESS  129617.00   3608300.00      43   36083.00
           OMSN       SUCCESS    4470.00    149000.00       2       0.00
           TMONEYTG   SUCCESS  165417.44   5002970.00     551   40023.76
Total                         3776036.82 115894673.00    6762 1702159.23

In [24]:
maj = dfpmt.loc[(dfpmt['Statut'] == 'PENDING') & (dfpmt['CINETPAY'] == 1)]
maj

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Montant,Fee amount,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre
15158,2024-12-16 16:27:13,NaN,OM,COASTWIN SARLU CI,0d60e712-7fb0-4290-bdeb-de11c98b2e55,3049030703,NaN,NaN,7500.00,150.00,...,2024-12-16,VAL,OM,CI241216.1628.B80292,60.00,90.00,1,0,0,7500.00
16006,2024-12-16 23:43:22,NaN,OM_BF,1XBET BF,803044b3-4f55-4f21-a1d6-43f398d892bd,3049908773,NaN,NaN,7500.00,450.00,...,2024-12-16,VAL,OMBF,CI241217.0012.95788587,75.00,375.00,1,0,0,7500.00
16016,2024-12-16 23:43:24,NaN,MOOV_BF,Bet Winner BF,57d7f01a-3c19-4246-85df-1b0cdb2f7a1c,160485958,NaN,NaN,20000.00,1100.00,...,2024-12-16,VAL,MOOVBF,CI241217.0007.F00057,200.00,900.00,1,0,0,20000.00
16046,2024-12-16 23:44:08,NaN,OM_BF,Bet Winner BF,d203b9ce-212a-413f-900a-fe79cd046051,160486052,NaN,NaN,1000.00,55.00,...,2024-12-16,VAL,OMBF,CI241217.0012.10392235,10.00,45.00,1,0,0,1000.00
16068,2024-12-16 23:44:39,NaN,OM_BF,Bet Winner BF,564df386-46ec-43b0-b05e-20c97a945d19,160486084,NaN,NaN,3000.00,165.00,...,2024-12-16,VAL,OMBF,CI241217.0012.86795670,30.00,135.00,1,0,0,3000.00
16087,2024-12-16 23:45:30,NaN,OM_BF,1XBET BF,6d0bb547-60c9-4be9-9c21-5687d934812b,3049909833,NaN,NaN,15500.00,930.00,...,2024-12-16,VAL,OMBF,CI241217.0013.95788614,155.00,775.00,1,0,0,15500.00


In [25]:
perte = dfpmt.loc[(dfpmt['Statut'] == 'FAILED') & (dfpmt['CINETPAY'] == 1)]
perte

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Montant,Fee amount,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre


In [26]:
trx_succes_a_rechercher = dfop.loc[(dfop['PMT'] == 0)]
trx_succes_a_rechercher

,Date Création (GMT),Date Modification,Date de traitement,ID CinetPay,ID Marchand,ID Operateur,Téléphone,Opérateur,Montant Envoyé,Marchand verifié ?,Statut,Commentaire,DateCourte,PMT,Nombre


In [27]:
result_trx_succes_a_rech = dfpmt.loc[(dfpmt['Transaction ID'] == '7606e7a9-9532-4c89-a004-6177f942362b')]
result_trx_succes_a_rech

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Montant,Fee amount,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre


In [28]:
trx_pending_abs_cinet = dfpmt.loc[(dfpmt['Statut'] == 'PENDING') & (dfpmt['CINETPAY'] == 0)]
trx_pending_abs_cinet.shape

(9, 29)

In [29]:
trx_succes_abs_cinet = dfpmt.loc[(dfpmt['Statut'] == 'SUCCESS') & (dfpmt['CINETPAY'] == 0)]
trx_succes_abs_cinet.shape

(12098, 29)

In [30]:
 # Enregistrer les données filtrées dans un fichier Excel

#output_file_path = 'D:\RECO\CINETPAY\PAYOUT\MAJ/trx_a_mettre_ajour.xlsx'
#maj.to_excel(output_file_path, index=False)

#print(f"Fichier enregistré sous : {output_file_path}")


In [31]:
report_date = "20241216"
# Générer automatiquement le chemin de sortie avec la date actuelle
current_date = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = 'D:/RECO/CINETPAY/PAYOUT/REACPANALYSE/DEC/'
output_file_name = f"{current_date}_RAPPORT_ANALYSE_{report_date}.xlsx"
output_file_path = os.path.join(output_dir, output_file_name)
# Chemin du fichier de sortie

# Création de DataFrames d'exemple
data_pmt = dfpmt
data_op = dfop
REC_NEW=new_rec
REJ=rej
pertes = perte
maj_jour_pending_a_success = maj
trxenattente_abs = trx_pending_abs_cinet
trx_succes_abs_cinetpay = trx_succes_abs_cinet
trx_cinetpay_succes_abs_pmt = trx_succes_a_rechercher
# Dictionnaire de DataFrames
feuilles = {
    'DATA PMT': data_pmt,
    'data op': data_op,
    'Rec_New' :REC_NEW,
    'rej' : REJ,
    'pertes': pertes,
    'maj_pending' : maj_jour_pending_a_success,
    'trxenattente_abs': trx_pending_abs_cinet,
    'trx_succes_pmt_abs_cinet' :trx_succes_abs_cinetpay,
    'trx_succes_cinetpay_abs_pmt' :trx_cinetpay_succes_abs_pmt
}

# Création de l'objet ExcelWriter
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    for nom_feuille, df in feuilles.items():
        df.to_excel(writer, sheet_name=nom_feuille, index=False)

print(f"Fichier Excel avec plusieurs feuilles sauvegardé avec succès: {output_file_path}")

Fichier Excel avec plusieurs feuilles sauvegardé avec succès: D:/RECO/CINETPAY/PAYOUT/REACPANALYSE/DEC/20241217_091645_RAPPORT_ANALYSE_20241216.xlsx


In [32]:
select=dfpmt.groupby(['Statut','DateCourte'])['Montant'].count()
select

Statut   DateCourte
FAILED   2024-12-15      211
         2024-12-16      100
PENDING  2024-12-15        1
         2024-12-16       14
SUCCESS  2024-12-15    12098
         2024-12-16     6756
Name: Montant, dtype: int64

In [33]:
trx = dfpmt.loc[(dfpmt['Statut'] == 'en attente') & (dfpmt['DateCourte'] == '2024-06-10')]
trx

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Montant,Fee amount,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre


ANALYSE PMT

In [34]:
succes= dfpmt[(dfpmt['Statut'] == 'SUCCESS')]
echec =dfpmt[(dfpmt['Statut'] == 'FAILED')]

In [35]:
success=succes[['Created Date',
           'Payment Date',
           'Operator',
           'Merchant Name',
           'Transaction ID',
           'Phone Number',
           'DateCourte',
           'DATEOP',
           'Statut',
           'STATUTOP',
           'OPERATOROP',
           'Montant'
]]
success.head(2)

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,Phone Number,DateCourte,DATEOP,Statut,STATUTOP,OPERATOROP,Montant
0,2024-12-15 08:53:31,NaN,OM_BF,Mel Bet BF,958530cc-c21f-4775-a8f2-4884702743f7,22675702212,2024-12-15,NaN,SUCCESS,NaN,NaN,25000.00
1,2024-12-15 15:42:21,NaN,OM_BF,1XBET BF,ec1617ee-2155-4dcc-8ccd-06569034b57b,22675920295,2024-12-15,NaN,SUCCESS,NaN,NaN,50000.00


In [36]:
echecs=echec[['Created Date',
           'Payment Date',
           'Operator',
           'Merchant Name',
           'Transaction ID',
           'Phone Number',
           'DateCourte',
           'DATEOP',
           'Statut',
           'STATUTOP',
           'OPERATOROP',
           'Montant'
]]
echecs.head(2)

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,Phone Number,DateCourte,DATEOP,Statut,STATUTOP,OPERATOROP,Montant
52,2024-12-15 08:54:31,NaN,OM_CD,Betwinner CD,d8c068f6-3719-4ea3-b39d-c3a6329a9bb9,243910681174,2024-12-15,NaN,FAILED,NaN,NaN,8000.00
97,2024-12-15 12:24:57,NaN,OM_BF,1XBET BF,f9ed455a-0cde-4650-85ff-379d833a0239,22675712193,2024-12-15,NaN,FAILED,NaN,NaN,10100.00


In [37]:
select_success_min = success.groupby(['DateCourte','STATUTOP'])['Montant'].min()
select_success_min.reset_index()

,DateCourte,STATUTOP,Montant
0,2024-12-16,VAL,400.00


In [38]:
select_success_max = success.groupby(['DateCourte','STATUTOP'])['Montant'].max()
select_success_max.reset_index()

,DateCourte,STATUTOP,Montant
0,2024-12-16,VAL,1500000.00


In [39]:
select_echec_min = echecs.groupby(['DateCourte', 'STATUTOP'])['Montant'].min()
select_echec_min.reset_index()

,DateCourte,STATUTOP,Montant
0,2024-12-16,REJ,400.00


In [40]:
select_echec_max = echecs.groupby(['DateCourte', 'STATUTOP'])['Montant'].max()
select_echec_max.reset_index()

,DateCourte,STATUTOP,Montant
0,2024-12-16,REJ,400000.00


REQUETE SUPPORT

In [41]:
select_pays = df_filteredpmt.groupby(['Pays']).agg(
    Nombre =('Montant', 'count'),
    Volume =('Montant', 'sum')
)
select_pays

,Nombre,Volume
Pays,,
BF,3305,38568781.00
CD,931,47283558.00
CI,1766,23573519.00
SN,2,149000.00
TG,758,6319815.00


In [42]:
select_marchand=df_filteredpmt.groupby(['Merchant Name','Pays','Operator']).agg(
    Nombre=('Montant', 'count'),
    Volume_transaction=('Montant','sum')
)
select_marchand

Nombre  \
Merchant Name                          Pays Operator            
1XBET BF                               BF   MOOV_BF       500   
                                            OM_BF         307   
1XBET CD                               CD   AIRTEL_CD     427   
                                            MPESA_CD      243   
                                            OM_CD          25   
1XBET TG                               TG   MOOV_TG       121   
                                            TMONEY_TG     350   
AFROPARI BF                            BF   MOOV_BF        14   
                                            OM_BF          34   
AFROPARI CI                            CI   OM             57   
AFROPARI TG                            TG   TMONEY_TG       5   
AMCDRC LTD CI                          CI   OM            465   
AVEONET CI                             CI   OM              6   
BETMOMO BF                             BF   MOOV_BF         3   
                                            OM_BF          47   
BETMOMO CI                             CI   OM            386   
Bet Winner BF                          BF   MOOV_BF       422   
                                            OM_BF        1269   
Bet Winner SN                          SN   OM_SN           2   
BetWinner TG                           TG   MOOV_TG        13   
                                            TMONEY_TG      37   
Betwinner CD                           CD   AIRTEL_CD     106   
                                            MPESA_CD       58   
                                            OM_CD          30   
COASTWIN SARLU CI                      CI   OM            804   
Good Game                              CI   OM             34   
MFI Investment limited (LUCKY STAR) CI CI   OM              7   
Mel Bet BF                             BF   MOOV_BF       143   
                                            OM_BF         561   
Mel Bet TG                             TG   MOOV_TG        68   
                                            TMONEY_TG     163   
OTRADA HOSPITALITY SARL RDC            CD   AIRTEL_CD      26   
                                            MPESA_CD       11   
                                            OM_CD           5   
PARIPULSE BF                           BF   OM_BF           3   
Paleo Holdings LTD BF                  BF   OM_BF           2   
Paleo Holdings LTD CI                  CI   OM              7   
Paleo Holdings LTD TG                  TG   TMONEY_TG       1   

                                                       Volume_transaction  
Merchant Name                          Pays Operator                       
1XBET BF                               BF   MOOV_BF            6594009.00  
                                            OM_BF              4374575.00  
1XBET CD                               CD   AIRTEL_CD         13444611.00  
                                            MPESA_CD          20839518.00  
                                            OM_CD              2143800.00  
1XBET TG                               TG   MOOV_TG             592530.00  
                                            TMONEY_TG          2779305.00  
AFROPARI BF                            BF   MOOV_BF              40200.00  
                                            OM_BF               713760.00  
AFROPARI CI                            CI   OM                  464710.00  
AFROPARI TG                            TG   TMONEY_TG             6700.00  
AMCDRC LTD CI                          CI   OM                 6678641.00  
AVEONET CI                             CI   OM                   40200.00  
BETMOMO BF                             BF   MOOV_BF              18070.00  
                                            OM_BF              1236610.00  
BETMOMO CI                             CI   OM                 7201705.00  
Bet Winner BF                          BF   MOOV_BF            4168299.00  
          

In [43]:
requete_support =dfpmt.loc[(dfpmt['Transaction ID'] =='253f77e6-31d3-45d2-be70-a61ba3f601aa')]
requete_support

,Created Date,Payment Date,Operator,Merchant Name,Transaction ID,merchant_transaction_id,ID Opérateur,external_transaction_id,Montant,Fee amount,...,DATEOP,STATUTOP,OPERATOROP,IDOPERATOR,frais_op,Frais_pmt,CINETPAY,NEW_REC,REJ,Nombre


In [44]:
select_country=df_filteredpmt.groupby(['Pays']).agg(
    Nombre=('Montant', 'count'),
    Volume_transaction=('Montant','sum')
)
select_country

,Nombre,Volume_transaction
Pays,,
BF,3305,38568781.00
CD,931,47283558.00
CI,1766,23573519.00
SN,2,149000.00
TG,758,6319815.00


In [47]:
select_date=df_filteredpmt[(df_filteredpmt['DateCourte'] > "2024-12-16")]
select_country=select_date.groupby(['Pays']).agg(
    Nombre=('Montant', 'count'),
    Volume_transaction=('Montant','sum')
)
select_country

,Nombre,Volume_transaction
Pays,,


TRAITEMENT CHIFFRE AFFAIRE

In [67]:
#MISE EN PLACE DE RECHERCHE X POUR RECUPERATION CHEZ L'OP
        # Vérification des correspondances entre A1 et B1
correspondance_Montant_op= dfop.set_index('ID Marchand')['Montant Envoyé']

df_filteredpmt['MtOP'] = df_filteredpmt['Transaction ID'].map(correspondance_Montant_op)

In [68]:
df_filteredpmt['taux_marchand']=df_filteredpmt['Fee amount'] / df_filteredpmt['Montant']
df_filteredpmt['Frais_op']= 0
df_filteredpmt['Tauxopp']= 0
df_filteredpmt['type']='Payout'
df_filteredpmt['Comm_op']=df_filteredpmt['Montant'] * df_filteredpmt['Tauxopp']
df_filteredpmt['Comm_ttc']=df_filteredpmt['Fee amount'] - df_filteredpmt['Comm_op']

In [69]:
chiffre_affaire_pmt = df_filteredpmt.rename(columns={
        'DateCourte': 'Date Creation',
        'Transaction ID': 'ID Transaction',
        'Pays': 'country',
        'Merchant Name': 'merchant_name',
        'provider_name': 'provider_name',
        'OPERATOROP': 'operator',
        'Statut': 'statut',
        'Montant': 'montant_transactions',
        'Fee amount':'fee_amount',
        'taux_marchand':'Taux Marchand',
        'DATEOP':'Date OP',
        'IDOPERATOR':'ID OP',
        'MtOP':'Montant OP',
        'Frais_op':'Frais OP',
        'type':'Type',
        'Tauxopp':'Taux Opérateur',
        'Comm_op':'Commission OP',
        'Comm_ttc':'Commission TTC'
    })

In [70]:
chiffre_affaire=chiffre_affaire_pmt[['Date Creation',
                                     'ID Transaction',
                                     'country',
                                     'merchant_name',
                                     'provider_name',
                                     'operator',
                                     'statut',
                                     'montant_transactions',
                                     'fee_amount',
                                     'Taux Marchand',
                                     'Date OP',
                                     'ID OP',
                                     'Montant OP',
                                     'Frais OP',
                                     'Type',
                                     'Taux Opérateur',
                                     'Commission OP',
                                     'Commission TTC'
]]  

In [71]:
chiffre_affaire

,Date Creation,ID Transaction,country,merchant_name,provider_name,operator,statut,montant_transactions,fee_amount,Taux Marchand,Date OP,ID OP,Montant OP,Frais OP,Type,Taux Opérateur,Commission OP,Commission TTC
7254,2024-12-16,cfe7eca5-f4f8-489a-9ca3-73bdfce104ba,BF,Bet Winner BF,CinetPay_Payout_OM_BF,OMBF,SUCCESS,5000.00,275.00,0.06,2024-12-16,CI241216.0634.42177728,5000,0,Payout,0,0.00,275.00
7257,2024-12-16,aef222ea-a0dd-45fb-bd2f-fe36eb80f0a2,CI,AMCDRC LTD CI,CinetPay_Payout_OM_CI,OM,SUCCESS,5000.00,450.00,0.09,2024-12-16,CI241216.0634.B69126,5000,0,Payout,0,0.00,450.00
7258,2024-12-16,f76ccfa5-4c00-4133-8a35-5a90b02753fe,BF,Bet Winner BF,CinetPay_Payout_OM_BF,OMBF,SUCCESS,1000.00,55.00,0.06,2024-12-16,CI241216.0858.68144660,1000,0,Payout,0,0.00,55.00
7270,2024-12-16,e98907e9-8864-4760-8904-da689f4533cf,TG,Mel Bet TG,CinetPay_Payout_TMONEY_TG,TMONEYTG,SUCCESS,2000.00,60.00,0.03,2024-12-16,8181513008,2000,0,Payout,0,0.00,60.00
7272,2024-12-16,e3f14eb2-c187-47ac-82b3-f0cdc684eb26,BF,Bet Winner BF,CinetPay_Payout_MOOV_BF,MOOVBF,SUCCESS,2000.00,110.00,0.06,2024-12-16,CI241216.0855.F09076,2000,0,Payout,0,0.00,110.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19175,2024-12-16,376f61ad-add5-4180-b9bf-42f63c523a18,TG,1XBET TG,CinetPay_Payout_MOOV_TG,FLOOZTG,SUCCESS,1100.00,55.00,0.05,2024-12-16,2241216367898,1100,0,Payout,0,0.00,55.00
19176,2024-12-16,0ceb4b3f-042d-413f-bbff-554975c1b6c9,BF,1XBET BF,CinetPay_Payout_OM_BF,OMBF,SUCCESS,23000.00,1380.00,0.06,2024-12-16,CI241216.2322.68114413,23000,0,Payout,0,0.00,1380.00
19177,2024-12-16,3f5210db-da6c-4ea6-9020-ed53ff9f309f,CD,Betwinner CD,CinetPay_Payout_AIRTEL_CD,AIRTELCD,SUCCESS,10000.00,400.00,0.04,2024-12-16,CI241216.1716.D13955,10000,0,Payout,0,0.00,400.00
19178,2024-12-16,788e5b6b-6ea0-4d95-8ca9-7767b28113f3,BF,Mel Bet BF,CinetPay_Payout_OM_BF,OMBF,SUCCESS,1050.00,31.50,0.03,2024-12-16,CI241216.2042.68161662,1050,0,Payout,0,0.00,31.50


In [66]:
report_date = "20241216"
# Générer automatiquement le chemin de sortie avec la date actuelle
current_date = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = 'D:\RECONCILIATIONS\TRANSFERT\Cinetpay\CA\DEC'
output_file_name = f"{current_date}_RAPPORT_CA_CINETPAY_PAYOUT_{report_date}.xlsx"
output_file_path = os.path.join(output_dir, output_file_name)

# Création de DataFrames d'exemple
CA = chiffre_affaire


# Dictionnaire de DataFrames
feuilles = {
    'Chiffre_Affaire': CA,
    
}

# Création de l'objet ExcelWriter
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    for nom_feuille, df in feuilles.items():
        df.to_excel(writer, sheet_name=nom_feuille, index=False)

print(f"Fichier Excel avec plusieurs feuilles sauvegardé avec succès: {output_file_path}")

Fichier Excel avec plusieurs feuilles sauvegardé avec succès: D:\RECONCILIATIONS\TRANSFERT\Cinetpay\CA\DEC\20241217_143048_RAPPORT_CA_CINETPAY_PAYOUT_20241216.xlsx
